## Notebook to run *cis* correlation analysis between gene expression and ATAC peaks

only interest in gene's with a statistically significant age effect and the ATAC peaks that are <i>cis</i> proximal

In [ ]:
!date

#### import libraries

In [ ]:
from scanpy import read_h5ad
from pandas import DataFrame as PandasDF, read_parquet, read_csv, concat
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
from os.path import exists
import numpy as np
from seaborn import barplot, lmplot, scatterplot
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#### set notebook variables

In [ ]:
# parameters
endogenous = 'GEX'
exogenous = 'ATAC'
category = 'curated_type' # 'curated_type' for broad and 'cluster_name' for specific
REGRESSION_TYPE = 'glm_tweedie'

In [ ]:
# parameters
project = 'aging_phase2'
if category == 'curated_type':
    prefix_type = 'broad'
elif category == 'cluster_name':
    prefix_type = 'specific' 

# directories
wrk_dir = '/labshare/raph/datasets/adrd_neuro/brain_aging/phase2'
results_dir = f'{wrk_dir}/results'
quants_dir = f'{wrk_dir}/quants'
figures_dir = f'{wrk_dir}/figures'

# in files
endo_results_file = f'{results_dir}/{project}.{endogenous}.{prefix_type}.{REGRESSION_TYPE}_fdr_filtered.age.csv'
exo_results_file = f'{results_dir}/{project}.{exogenous}.{prefix_type}.{REGRESSION_TYPE}_fdr_filtered.age.csv'
anndata_file = f'{quants_dir}/{project}.multivi.curated_final.h5ad'

# out files
results_file = f'{results_dir}/{project}.{endogenous}-{exogenous}.{prefix_type}.{REGRESSION_TYPE}.cis.parquet'
results_fdr_file = f'{results_dir}/{project}.{endogenous}-{exogenous}.{prefix_type}.{REGRESSION_TYPE}_fdr.cis.csv'

# constants
DEBUG = True
MAX_DIST = 1_000_000
ALPHA = 0.05
covariate_terms = ['sex', 'ancestry', 'pmi', 'ph', 'smoker', 'bmi', 'gex_pool', 'atac_pool']
covar_term_formula = ' + '.join(covariate_terms)
if DEBUG:
    print(covar_term_formula)
    print(results_file)
    print(results_fdr_file)

#### functions

In [ ]:
def load_quantification(cell_name: str, modality: str, verbose: bool=False) -> PandasDF:
    this_file = f'{quants_dir}/{project}.{modality}.{prefix_type}.{cell_name}.pb.parquet'
    if not exists(this_file):
        return None
    df = read_parquet(this_file)
    if verbose:
        print(f'shape of read {cell_name} quantifications {df.shape}')        
        display(df.sample(5))
    return df

def glm_model(formula: str, df: PandasDF, model_type: str='rlm'):
    if model_type == 'glm_tweedie':
        model = smf.glm(formula=formula, data=df, 
                        family=sm.families.Tweedie(link=sm.families.links.log(), 
                                                   var_power=1.6, 
                                                   eql=True))
    elif model_type == 'rlm':
        model = smf.rlm(formula=formula, data=df)        
    elif model_type == 'glm':
        model = smf.glm(formula=formula, data=df)        
    result = model.fit()
    return result

def cis_correlation(df: PandasDF, endo_term: str, exog_term: str, verbose: bool=False) -> tuple:
    model_terms = [endo_term, exog_term] + covariate_terms + ['cell_count_endo', 'cell_count_exog']
    this_formula = f'Q("{endo_term}") ~ Q("{exog_term}") + {covar_term_formula} + cell_count_endo + cell_count_exog'
    try:
        # run GLM via statsmodel
        result = glm_model(this_formula, df[model_terms], model_type=REGRESSION_TYPE)
        ret_exog_term = f'Q("{exog_term}")'
        ret_list = [endo_term, exog_term, result.params['Intercept'], 
                    result.params[ret_exog_term], result.bse[ret_exog_term], 
                    result.tvalues[ret_exog_term], result.pvalues[ret_exog_term]]
        if verbose:
            print(f'df shape {df.shape}')
            print(result.summary())
            print(['endo_feature', 'exog_feature', 'intercept', 'coef', 'stderr', 'z', 'p-value'])
            print(ret_list)
    except:
#         print(f'Caught Error for {endo_term}')
        ret_list = [endo_term, exog_term] + [np.nan] * 5
  
    return ret_list


def compute_bh_fdr(df: PandasDF, alpha: float=0.05, p_col: str='p-value',
                   method: str='fdr_bh', verbose: bool=True) -> PandasDF:
    ret_df = df.copy()
    test_adjust = multipletests(np.array(ret_df[p_col]), alpha=alpha, 
                                method=method)
    ret_df[method] = test_adjust[1]
    if verbose:
        print(f'total significant after correction: {ret_df.loc[ret_df[method] < alpha].shape}')
    return ret_df

def load_tissue_quants(tissue: str, endo_ids: set, exog_ids: set, 
                       verbose: bool=False) -> {PandasDF, PandasDF}: 
    endo_data = load_quantification(tissue, endogenous)
    exog_data = load_quantification(tissue, exogenous)
    if verbose:
        print(f'shape of endogenous data {endo_data.shape}')
        print(f'shape of exogenous data {exog_data.shape}')
    # subset to only needed endo and exog features
    endo_data = endo_data[list(set(endo_data.columns) & endo_ids) + ['cell_count']]
    exog_data = exog_data[list(set(exog_data.columns) & exog_ids) + ['cell_count']]
    if verbose:
        print(f'shape of subset endogenous data {endo_data.shape}')
        print(f'shape of subset exogenous data {exog_data.shape}')        
        display(endo_data.sample(4))
        display(exog_data.sample(4))
    return endo_data, exog_data

def merge_analysis_data(endo_data: PandasDF, exog_data: PandasDF, covars_df: PandasDF, 
                        endo_ids: set, exog_ids: set, verbose: bool=False) -> PandasDF:
    # subset to only needed endo and exog features
    endo_data = endo_data[list(set(endo_data.columns) & endo_ids) + ['cell_count']]
    exog_data = exog_data[list(set(exog_data.columns) & exog_ids) + ['cell_count']]
    tissue_data = (endo_data.merge(exog_data, how='inner', left_index=True, right_index=True, suffixes=('_endo', '_exog'))
                   .merge(covars_df, how='inner', left_index=True, right_index=True))
    if verbose:
        print(f'shape of merged data is {tissue_data.shape}')        
        display(tissue_data.sample(5))
    return tissue_data

def show_pair(tissue: str, covars_df: PandasDF, endo_id: str, exog_id: str,
              verbose: bool=True):
    # this weird set from single ID is just so I can re-use same functions
    endo_ids = set([endo_id])
    exo_ids = set([exog_id])    
    # load quants data
    endo_data, exog_data = load_tissue_quants(tissue, endo_ids, exo_ids, verbose)
    # merge data source
    tissue_data = merge_analysis_data(endo_data, exog_data, covars_df, endo_ids, exo_ids, verbose)
    # run the regressions
    cis_correlation(tissue_data, endo_id, exog_id, verbose)
    # plot the pair
    with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 50}):
        plt.style.use('seaborn-v0_8-talk')
        lmplot(x=exog_id,y=endo_id, data=tissue_data, palette='Purples')
        plt.title(f'{endo_id} ~ {exog_id}', fontsize='large') 
        plt.xlabel(exog_id)
        plt.ylabel(endo_id)        
        plt.show()

def volcano_plot(df: PandasDF, x_term: str='coef', y_term: str='p-value', 
                 alpha: float=0.05, adj_p_col: str='fdr_bh', title: str=None, 
                 filter_nseeff: bool=True, extreme_size: float=10.0):
    df = df.copy()
    df = df.reset_index(drop=True)    
    if filter_nseeff:
        df = df.loc[((-extreme_size < df[x_term]) & 
                    (df[x_term] < extreme_size) &
                    (~df['z'].isna()) | 
                    (df[adj_p_col] < alpha))]
    plt.figure(figsize=(9,9))
    log_pvalue = -np.log10(df[y_term])
    is_sig = df[adj_p_col] < alpha
    with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 50}):
        plt.style.use('seaborn-v0_8-talk')    
        scatterplot(x=x_term, y=log_pvalue, data=df, hue=is_sig, palette='Purples')
        plt.title(title)
        plt.xlabel('effect')
        plt.ylabel('-log10(p-value)')
        fig_file = f'{figures_dir}/{project}.{endogenous}-{exogenous}.{prefix_type}.{REGRESSION_TYPE}_volcano.{title}.png'
        plt.savefig(fig_file)
        plt.show()

### load the GEX results to find which genes in what 'cell-types' should be interegated

In [ ]:
endo_results_df = read_csv(endo_results_file)
print(f'shape of GEX results {endo_results_df.shape}')
if DEBUG:
    display(endo_results_df.sample(5))

#### how many genes per cell-type with a results will be considered

In [ ]:
print(endo_results_df.feature.nunique())
display(endo_results_df.tissue.value_counts())

### load the GEX ~ ATAC results and combine

In [ ]:
%%time

cis_results = None
print(f'### {prefix_type}')    
for cell_type in endo_results_df.tissue.unique():
    print(f'--- {cell_type}')
    in_file = f'{results_dir}/{project}.{endogenous}-{exogenous}.{prefix_type}.{cell_type}.{REGRESSION_TYPE}.cis.csv'
    if exists(in_file):
        this_result = read_csv(in_file)
        this_result['type'] = prefix_type
        this_result['tissue'] = cell_type
        cis_results = concat([cis_results, this_result])

In [ ]:
print(f'shape of all load results {cis_results.shape}')
if DEBUG:
    display(cis_results.type.value_counts())
    display(cis_results.groupby('type').tissue.value_counts())    
    display(cis_results.sample(5))

#### compute the FDR values

In [ ]:
cis_results['p-value'] = cis_results['p-value'].fillna(1)
cis_results = compute_bh_fdr(cis_results)
print(cis_results.shape)
if DEBUG:
    display(cis_results.sort_values('fdr_bh').head())

### save the combined results

In [ ]:
%%time
cis_results.to_parquet(results_file, index=False)
cis_results.loc[cis_results.fdr_bh <= ALPHA].to_csv(results_fdr_file, index=False)

### count of significant GEX~ATAC pairs by broad curated cell-type

In [ ]:
print(cis_results.loc[cis_results.fdr_bh <= ALPHA]['tissue'].nunique())
display(cis_results.loc[cis_results.fdr_bh <= ALPHA].groupby('type').tissue.value_counts())

In [ ]:
display(cis_results.loc[cis_results.fdr_bh <= ALPHA].groupby('tissue').endo_feature.value_counts().nlargest(10))
display(cis_results.loc[cis_results.fdr_bh <= ALPHA].groupby('tissue').exog_feature.value_counts().nlargest(10))

In [ ]:
display(cis_results.loc[cis_results.fdr_bh <= ALPHA].groupby(['endo_feature', 'exog_feature']).tissue.value_counts().nlargest(10))

In [ ]:
display(cis_results.loc[cis_results.fdr_bh <= ALPHA].groupby(['endo_feature']).exog_feature.value_counts().nlargest(5))

In [ ]:
cis_results.loc[(cis_results.fdr_bh <= ALPHA) & (cis_results.endo_feature == 'YBX3')].tissue.value_counts()

### volcano plots

In [ ]:
print(f'### {prefix_type}')    
for cell_type in endo_results_df.tissue.unique():
    print(f'--- {cell_type}')
    specific_results = cis_results.loc[cis_results.tissue == cell_type]
    if specific_results.shape[0] > 0:
        volcano_plot(specific_results, title=cell_type)
    else:
        print('no results to plot')

### per tissue what percentage of the age associated GEX features have cis correlated ATAC

In [ ]:
gex_proportions = {}
sig_results = cis_results.loc[cis_results.fdr_bh <= ALPHA]
for cell_type in endo_results_df.tissue.unique():
    tissue_age_results = endo_results_df.loc[endo_results_df.tissue == cell_type]
    tissue_cis_results = (sig_results.loc[(sig_results.tissue == cell_type) 
                          & (sig_results.endo_feature.isin(tissue_age_results.feature))])
    proportion = round((tissue_cis_results.endo_feature.nunique()/tissue_age_results.feature.nunique()) * 100, 1)
    print(cell_type, proportion)
    gex_proportions[cell_type] = proportion
gex_prop_df = PandasDF(data=list(gex_proportions.items()), columns=['tissue', 'percentage'])
if DEBUG:
    display(gex_prop_df)    

### load the ATAC results

In [ ]:
exo_results_df = read_csv(exo_results_file)
print(f'shape of ATAC results {exo_results_df.shape}')
if DEBUG:
    display(exo_results_df.sample(5))

#### how many peaks per cell-type

In [ ]:
print(exo_results_df.feature.nunique())
display(exo_results_df.tissue.value_counts())

### per tissue what percentage of the age associated GEX features have cis correlated age associated ATAC feature

In [ ]:
gex_age_atac_proportions = {}
sig_results = cis_results.loc[cis_results.fdr_bh <= ALPHA]
for cell_type in endo_results_df.tissue.unique():
    tissue_gex_age_results = endo_results_df.loc[endo_results_df.tissue == cell_type]
    tissue_atac_age_results = exo_results_df.loc[exo_results_df.tissue == cell_type]    
    tissue_cis_results = (sig_results.loc[(sig_results.tissue == cell_type) 
                          & (sig_results.exog_feature.isin(tissue_atac_age_results.feature)) 
                          & (sig_results.endo_feature.isin(tissue_gex_age_results.feature))])
    if tissue_cis_results.endo_feature.nunique() > 0:
        proportion = round((tissue_cis_results.endo_feature.nunique()/tissue_gex_age_results.feature.nunique()) * 100, 1)
    else:
        proportion = 0.0
    print(cell_type, proportion)
    gex_age_atac_proportions[cell_type] = proportion
gex_atac_prop_df = PandasDF(data=list(gex_age_atac_proportions.items()), columns=['tissue', 'percentage'])
if DEBUG:
    display(gex_atac_prop_df)

### combined barplot of percentage of age associated genes with cis correlated ATAC peak(s)

In [ ]:
gex_prop_df['condition'] = 'All cis correlated peaks'
gex_atac_prop_df['condition'] = 'Age associated cis correlated peaks'
prop_df = concat([gex_prop_df, gex_atac_prop_df])

with rc_context({'figure.figsize': (9, 9), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-talk')
    barplot(data=prop_df.sort_values('percentage', ascending=False), 
            x='tissue', y='percentage', hue='condition', palette='Purples')
    plt.title('Percentage of age associated genes with correlated cis ATAC peak(s)')
    plt.xlabel('cell type')
    plt.ylabel('% of age associated genes')

if DEBUG:
    display(prop_df)

### take a look at some specific results

need to reload the data to do so

#### load the anndata file
easy and combine place to get genomic location of possible features; from the var

In [ ]:
adata = read_h5ad(anndata_file)
print(adata)
if DEBUG:
    display(adata.var.loc[adata.var.modality == 'Gene Expression'].sample(4))
    display(adata.var.loc[adata.var.modality == 'Peaks'].sample(4))

#### format sample covariates
sex, ancestry, age, (gex_pool or atac_pool), pmi, ph, smoker, bmi

In [ ]:
keep_terms = ['sample_id','sex', 'ancestry', 'age', 'gex_pool', 'atac_pool', 
              'pmi', 'ph', 'smoker', 'bmi']
covars_df = adata.obs[keep_terms].drop_duplicates().reset_index(drop=True)
covars_df = covars_df.set_index('sample_id')

if DEBUG:
    print(covars_df.shape)
    display(covars_df.head())
    display(covars_df.info())
    display(covars_df.smoker.value_counts())
    display(covars_df.bmi.describe())

##### fill any missing covariate terms
looks like smoker and bmi is missing for one sample will set it to mean of those values

In [ ]:
# fill the missing smoker and bmi value
covars_df.loc[covars_df.smoker.isna(), 'smoker'] = covars_df.smoker.mean().round(1)
covars_df.loc[covars_df.bmi.isna(), 'bmi'] = covars_df.bmi.mean().round(1)

if DEBUG:
    print(covars_df.shape)
    display(covars_df.info())
    display(covars_df.smoker.value_counts())
    display(covars_df.bmi.describe())

#### max significant by p-value result

In [ ]:
this_results = cis_results.loc[cis_results['p-value'] == min(cis_results['p-value'])]
this_hit = this_results.sort_values(by=['coef'], ascending=False).iloc[0]
print(this_hit)

In [ ]:
%%time
show_pair(this_hit.tissue, covars_df, this_hit.endo_feature, this_hit.exog_feature)

#### max significant by coef

In [ ]:
sig_results = cis_results.loc[cis_results.fdr_bh <= ALPHA]
this_results = sig_results.loc[sig_results.coef == max(sig_results.coef)]
this_hit = this_results.sort_values(by=['coef'], ascending=False).iloc[0]
print(this_hit)

In [ ]:
%%time
show_pair(this_hit.tissue, covars_df, this_hit.endo_feature, this_hit.exog_feature)

#### min significant by coef

In [ ]:
sig_results = cis_results.loc[cis_results.fdr_bh <= ALPHA]
this_results = sig_results.loc[sig_results.coef == min(sig_results.coef)]
this_hit = this_results.sort_values(by=['coef'], ascending=False).iloc[0]
print(this_hit)

In [ ]:
%%time
show_pair(this_hit.tissue, covars_df, this_hit.endo_feature, this_hit.exog_feature)

#### random

In [ ]:
this_hit = sig_results.sample().iloc[0]
print(this_hit)

In [ ]:
%%time
show_pair(this_hit.tissue, covars_df, this_hit.endo_feature, this_hit.exog_feature)

#### max non-significat by coef

In [ ]:
nonsig_results = cis_results.loc[(cis_results.fdr_bh > ALPHA) & 
                                 (~cis_results.z.isna())]
this_results = nonsig_results.loc[nonsig_results.coef == max(nonsig_results.coef)]
this_hit = this_results.iloc[0]
print(this_hit)

In [ ]:
%%time
show_pair(this_hit.tissue, covars_df, this_hit.endo_feature, this_hit.exog_feature)

In [ ]:
!date